In [96]:
import QuantLib as ql
from datetime import datetime as dt

https://www.implementingquantlib.com/2024/05/inflation-curves.html

## Assumptions

### Date

In [97]:
today = dt.today()
today = ql.Date(today.day, today.month, today.year)
ql.Settings.instance().evaluationDate = today

calendar = ql.Poland()
day_count = ql.ActualActual(ql.ActualActual.Bond)

### Bond

In [98]:
issue_date = ql.Date(25, 8, 2023)
maturity_date = ql.Date(25, 8, 2036)
settlement_days = 2
face_value = 1000.0
fixing_lag = ql.Period(3, ql.Months)
fixing_frequency = ql.Monthly
forecast_frequency = ql.Annual
coupon_rate = 0.02

## Inflation curve and index

In [99]:
# forecasts of the inflation
dates = [
    ql.Date(1, 1, 2024),
    ql.Date(1, 1, 2025),
    ql.Date(1, 1, 2026),
    ql.Date(1, 1, 2027),
    ql.Date(1, 1, 2028),
    ql.Date(1, 1, 2029),
    ql.Date(1, 1, 2030),
    ql.Date(1, 1, 2031),
    ql.Date(1, 1, 2032),
    ql.Date(1, 1, 2033),
    ql.Date(1, 1, 2034),
    ql.Date(1, 1, 2035),
    ql.Date(1, 1, 2036),
]
rates = [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]

inflation_curve_handle = ql.ZeroInflationTermStructureHandle(
    ql.ZeroInflationCurve(today, calendar, day_count, fixing_lag, forecast_frequency, dates, rates)
)

# Define custom CPI index
cpi_index = ql.ZeroInflationIndex(
    "CPI", ql.CustomRegion('Poland', 'PL'), False, fixing_frequency, fixing_lag, ql.PLNCurrency(), inflation_curve_handle
)

In [100]:
historical_cpi_fixings = [
    (ql.Date(1, 6, 2024), 0.001),
    (ql.Date(1, 5, 2024), 0.001),
    (ql.Date(1, 4, 2024), 0.011),
    (ql.Date(1, 3, 2024), 0.002),
    (ql.Date(1, 2, 2024), 0.003),
    (ql.Date(1, 1, 2024), 0.004),
    (ql.Date(1, 12, 2023), 0.001),
    (ql.Date(1, 11, 2023), 0.007),
    (ql.Date(1, 10, 2023), 0.003),
    (ql.Date(1, 9, 2023), -0.004),
    (ql.Date(1, 8, 2023), 0.000),
    (ql.Date(1, 7, 2023), -0.002),
    (ql.Date(1, 6, 2023), 0.000),
    (ql.Date(1, 5, 2023), 0.000),
    (ql.Date(1, 4, 2023), 0.007),
    (ql.Date(1, 3, 2023), 0.011),
    (ql.Date(1, 2, 2023), 0.012),
    (ql.Date(1, 1, 2023), 0.024),
]

for date, rate in historical_cpi_fixings:
    cpi_index.addFixing(date, rate)

## Bond

In [101]:
# Define the inflation-linked bond
issue_date = ql.Date(25, 8, 2023)
maturity_date = ql.Date(25, 8, 2036)
schedule = ql.Schedule(
    issue_date,
    maturity_date,
    ql.Period(ql.Annual),
    calendar,
    ql.Unadjusted,
    ql.Unadjusted,
    ql.DateGeneration.Forward,
    False,
)
schedule.dates()

(Date(25,8,2023),
 Date(25,8,2024),
 Date(25,8,2025),
 Date(25,8,2026),
 Date(25,8,2027),
 Date(25,8,2028),
 Date(25,8,2029),
 Date(25,8,2030),
 Date(25,8,2031),
 Date(25,8,2032),
 Date(25,8,2033),
 Date(25,8,2034),
 Date(25,8,2035),
 Date(25,8,2036))

In [102]:
growth_only = False
interpolation = ql.CPI.Linear

inflation_bond = ql.CPIBond(
    settlement_days,
    face_value,
    growth_only,
    cpi_index.fixing(today),  # base CPI
    fixing_lag,
    cpi_index,  # CPI index
    interpolation,
    schedule,  # schedule
    [coupon_rate],  # fixed rates
    day_count,  # day count convention
    ql.Unadjusted,  # business day convention
    issue_date,
    calendar,
)

In [103]:
# Create the yield term structure based on fixed rate bonds
yield_curve = [
    ("ON", 0.05740, today + ql.Period(1, ql.Days)),
    ("PS1024", 0.03929, ql.Date(25, 10, 2024)),
    ("PS0425", 0.04140, ql.Date(25, 4, 2025)),
    ("DS0725", 0.04524, ql.Date(25, 7, 2025)),
    ("DS0726", 0.04948, ql.Date(25, 7, 2026)),
    ("PS1026", 0.05000, ql.Date(25, 10, 2026)),
    ("PS0527", 0.05052, ql.Date(25, 5, 2027)),
    ("DS0727", 0.05048, ql.Date(25, 7, 2027)),
    ("WS0428", 0.05086, ql.Date(25, 4, 2028)),
    ("PS0728", 0.05130, ql.Date(25, 7, 2028)),
    ("WS0429", 0.05169, ql.Date(25, 4, 2029)),
    ("PS0729", 0.05150, ql.Date(25, 7, 2029)),
    ("DS1029", 0.05157, ql.Date(25, 10, 2029)),
    ("DS1030", 0.05233, ql.Date(25, 10, 2030)),
    ("DS0432", 0.05297, ql.Date(25, 4, 2032)),
    ("DS1033", 0.05339, ql.Date(25, 10, 2033)),
    ("DS1034", 0.05324, ql.Date(25, 10, 2034)),
    ("IZ0836", 0.03168, ql.Date(25, 8, 2036)),
    ("WS0437", 0.05408, ql.Date(25, 4, 2037)),
    ("WS0447", 0.05438, ql.Date(25, 4, 2047)),
]

interest_rate = ql.ForwardCurve(
    [point[2] for point in yield_curve], [point[1] for point in yield_curve], day_count, calendar
)

yield_curve = ql.YieldTermStructureHandle(interest_rate)

### Pricing

In [104]:
# Set up a pricing engine
bond_engine = ql.DiscountingBondEngine(yield_curve)
inflation_bond.setPricingEngine(bond_engine)

# Get the clean price
clean_price = inflation_bond.cleanPrice()
print(f"Clean Price: {clean_price:.2f}")

# Get the dirty price
dirty_price = inflation_bond.dirtyPrice()
print(f"Dirty Price: {dirty_price:.2f}")

# Get the yield
bond_yield = inflation_bond.bondYield(day_count, ql.Compounded, ql.Annual)
print(f"Yield: {bond_yield:.2%}")

# Get the NPV
npv = inflation_bond.NPV()
print(f"NPV: {npv:.2f}")

Clean Price: 90.54
Dirty Price: 91.01
Yield: 4.95%
NPV: 909.79


In [105]:
rate = ql.InterestRate(bond_yield, day_count, ql.Compounded, ql.Annual)
duration = ql.BondFunctions.duration(inflation_bond, rate, ql.Duration.Simple)
print(f'Duration: {duration:.2f}')

mduration = ql.BondFunctions.duration(inflation_bond, rate, ql.Duration.Modified)
print(f'Modified Duration: {mduration:.2f}')

Duration: 10.71
Modified Duration: 10.20


## Other

In [106]:
import pandas as pd

In [107]:
dates = [cf.date() for cf in inflation_bond.cashflows()]
cfs = [[cf.date().ISO(), cf.amount()] for cf in inflation_bond.cashflows()]
cfs

[['2024-08-25', 4.942574651006425],
 ['2025-08-25', 20.325211694357122],
 ['2026-08-25', 20.731715928244267],
 ['2027-08-25', 21.146350246809153],
 ['2028-08-25', 21.569277251745337],
 ['2029-08-25', 22.00066279678025],
 ['2030-08-25', 22.44067605271585],
 ['2031-08-25', 22.889489573770163],
 ['2032-08-25', 23.347279365245573],
 ['2033-08-25', 23.81422495255049],
 ['2034-08-25', 24.29050945160149],
 ['2035-08-25', 24.77631964063352],
 ['2036-08-25', 25.2718460334462],
 ['2036-08-25', 1263.5923016723098]]

In [108]:
df = pd.DataFrame(cfs)
df['discount'] = [yield_curve.discount(date) for date in dates]

df
df.to_csv("dane.csv", index=False, sep=";", decimal=",")